In [1]:
import pandas as pd

df = pd.read_csv('Derm1M_v2_pretrain.csv')

# 두 열만 선택
labels_df = df[['disease_label', 'hierarchical_disease_label']]

# 표시 옵션 설정 (긴 텍스트도 잘리지 않게)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 50)

# 확인
print(f"전체 행 수: {len(labels_df)}")
labels_df.head(30)

전체 행 수: 413210


,disease_label,hierarchical_disease_label
0,epidermolysis bullosa,"hereditary, epidermolysis bullosa"
1,no definitive diagnosis,no definitive diagnosis
2,"erythrasma, pitriasis versicolor, tenia capitis, trichophyton shonleni","erythrasma, pitriasis versicolor, tenia capitis, trichophyton shonleni"
3,ulcer,"reaction patterns and descriptive terms, ulcers, erosion of skin, ulcer"
4,nevus,nevus
5,herpes zoster,"inflammatory, infectious, viral, herpes zoster"
6,neonatal toxic erythema,neonatal toxic erythema
7,ashidermatosis,ashidermatosis
8,syringoma,"proliferations, benign, non-melanocytic, syringoma"
9,basal cell carcinoma,"proliferations, malignant, non-melanocytic, basal cell carcinoma"


In [ ]:
import pandas as pd
import json, re

# 정규화 함수: 양쪽 공백 제거 → 소문자 → 여러 공백을 단일 공백으로
norm_re = re.compile(r"\s+")
def norm(s: str) -> str:
    return norm_re.sub(" ", s.strip().lower())

# ontology.json 읽기
with open('ontology.json', 'r', encoding='utf-8') as f:
    ontology = json.load(f)

# 유효 노드 집합 만들기 (루트 제외), 정규화 적용
valid_nodes = set(ontology.keys())
for v in ontology.values():
    valid_nodes.update(v)
valid_nodes.discard('root')
valid_nodes_norm = {norm(n) for n in valid_nodes}

print(f"리프+중간 노드 개수: {len(valid_nodes_norm)}")

# CSV 파일 읽기
df = pd.read_csv('Derm1M_v2_pretrain.csv')

print(f"\n전체 데이터: {len(df)}개")

def check_in_ontology(disease_label):
    if pd.isna(disease_label):
        return False, [], []
    diseases = [d.strip() for d in str(disease_label).split(',') if d.strip()]
    in_valid = [norm(d) in valid_nodes_norm for d in diseases]
    return all(in_valid), diseases, in_valid

df['all_in_ontology'], df['diseases_list'], df['in_valid_list'] = zip(*df['disease_label'].apply(check_in_ontology))

# 통계
total = len(df)
in_ontology_count = df['all_in_ontology'].sum()
print(f"\n온톨로지에 있음: {in_ontology_count}개 ({in_ontology_count/total*100:.1f}%)")
print(f"온톨로지에 없음: {total - in_ontology_count}개 ({(total - in_ontology_count)/total*100:.1f}%)")

# 온톨로지에 있는 데이터만 필터링
df_in_ontology = df[df['all_in_ontology']].copy()
single = df_in_ontology[~df_in_ontology['disease_label'].str.contains(',', na=False)]
multi = df_in_ontology[df_in_ontology['disease_label'].str.contains(',', na=False)]

print(f"\n온톨로지에 있는 데이터:")
print(f"  - 단일 질환: {len(single)}개")
print(f"  - 복수 질환: {len(multi)}개")

# 결과 확인 (처음 50개)
pd.set_option('display.max_colwidth', None)
result_df = df[['disease_label', 'diseases_list', 'in_valid_list', 'all_in_ontology']].head(50)
result_df


리프+중간 노드 개수: 369

전체 데이터: 413210개

온톨로지에 있음: 198044개 (47.9%)
온톨로지에 없음: 215166개 (52.1%)

온톨로지에 있는 데이터:
  - 단일 질환: 162027개
  - 복수 질환: 36017개


,disease_label,diseases_list,in_valid_list,all_in_ontology
0,epidermolysis bullosa,[epidermolysis bullosa],[True],True
1,no definitive diagnosis,[no definitive diagnosis],[False],False
2,"erythrasma, pitriasis versicolor, tenia capitis, trichophyton shonleni","[erythrasma, pitriasis versicolor, tenia capitis, trichophyton shonleni]","[False, False, False, False]",False
3,ulcer,[ulcer],[True],True
4,nevus,[nevus],[False],False
5,herpes zoster,[herpes zoster],[True],True
6,neonatal toxic erythema,[neonatal toxic erythema],[False],False
7,ashidermatosis,[ashidermatosis],[False],False
8,syringoma,[syringoma],[True],True
9,basal cell carcinoma,[basal cell carcinoma],[True],True


In [ ]:
import pandas as pd
import json

# ontology.json 읽기
with open('ontology.json', 'r') as f:
    ontology = json.load(f)

# 리프 노드 + 중간 노드 (총 369개)
leaf_nodes = set([node for node, children in ontology.items() if children == [] and node != 'root'])
internal_nodes = set([node for node, children in ontology.items() if children != [] and node != 'root'])
valid_nodes = leaf_nodes | internal_nodes
valid_nodes_lower = set([node.lower() for node in valid_nodes])

print(f"전체 유효 노드 개수: {len(valid_nodes)}")

# CSV 파일 읽기
df = pd.read_csv('Derm1M_v2_pretrain.csv')

# 각 disease_label을 콤마로 분리 (단일값도 포함)
def check_in_ontology(disease_label):
    if pd.isna(disease_label):
        return False, []
    
    diseases = [d.strip() for d in str(disease_label).split(',')]
    in_valid = [d.lower() in valid_nodes_lower for d in diseases]
    return all(in_valid), diseases

df['in_ontology'], df['diseases_list'] = zip(*df['disease_label'].apply(check_in_ontology))

# 통계
total = len(df)
in_ontology_count = df['in_ontology'].sum()

print(f"\n전체 데이터: {total}개")
print(f"온톨로지에 있음: {in_ontology_count}개 ({in_ontology_count/total*100:.1f}%)")
print(f"온톨로지에 없음: {total - in_ontology_count}개 ({(total - in_ontology_count)/total*100:.1f}%)")

# 온톨로지에 있는 데이터만 필터링
df_filtered = df[df['in_ontology']].copy()
print(f"\n필터링된 데이터: {len(df_filtered)}개")

전체 유효 노드 개수: 369


ValueError: too many values to unpack (expected 2)

In [19]:
import json
from collections import deque

with open('ontology.json', 'r') as f:
    ontology = json.load(f)

def count_nodes_by_depth_unique(ontology):
    """중복 없이 각 노드의 깊이 계산 (가장 짧은 경로 기준)"""
    depth_map = {}  # 노드별 깊이 저장
    visited = set()
    queue = deque([('root', 0)])
    
    while queue:
        node, depth = queue.popleft()
        
        # 이미 방문한 노드는 건너뛰기 (더 짧은 경로를 이미 찾음)
        if node in visited:
            continue
        
        visited.add(node)
        if node != 'root':  # root는 제외
            depth_map[node] = depth
        
        if node in ontology:
            for child in ontology[node]:
                if child not in visited:
                    queue.append((child, depth + 1))
    
    return depth_map

depth_map = count_nodes_by_depth_unique(ontology)

# 깊이별 노드 수 계산
depth_counts = {}
for node, depth in depth_map.items():
    depth_counts[depth] = depth_counts.get(depth, 0) + 1

print(f"총 노드 수 (root 제외): {len(depth_map)}")
print("\n각 레벨별 노드 수:")
for depth in sorted(depth_counts.keys()):
    print(f"Level {depth}: {depth_counts[depth]}개")

print(f"\n전체 합계: {sum(depth_counts.values())}개")

# 리프 노드 확인
leaf_nodes = [node for node, children in ontology.items() if children == [] and node != 'root']
print(f"\n리프 노드 수: {len(leaf_nodes)}")

총 노드 수 (root 제외): 369

각 레벨별 노드 수:
Level 1: 7개
Level 2: 88개
Level 3: 107개
Level 4: 148개
Level 5: 19개

전체 합계: 369개

리프 노드 수: 317


In [14]:
import pandas as pd, json, re

# 정규화 함수
norm_re = re.compile(r"\s+")
def norm(s: str) -> str:
    return norm_re.sub(" ", s.strip().lower())

# ontology 로드 및 부모 맵
with open("ontology.json", "r", encoding="utf-8") as f:
    ont = json.load(f)

parent = {}
nodes = set(ont.keys())
for n, ch in ont.items():
    nodes.update(ch)
    for c in ch:
        parent[c] = n

# 정규화 매핑: normalized -> originals
norm_map = {}
for n in nodes:
    norm_map.setdefault(norm(n), set()).add(n)

# path helper (원본 노드명으로 받아서 root->node)
def path_to_root(node: str):
    if node not in parent and node not in ont:
        return []
    p = [node]
    while p[-1] in parent:
        p.append(parent[p[-1]])
    return list(reversed(p))

# 유효 노드 집합 (예외 포함)
valid_nodes = {n for n in nodes if n != "root"}
valid_nodes.add("no definitive diagnosis")
valid_nodes_norm = {norm(n) for n in valid_nodes}

# CSV 로드
df = pd.read_csv("Derm1M_v2_pretrain.csv")

def check_in_ontology(disease_label):
    if pd.isna(disease_label):
        return False
    diseases = [d.strip() for d in str(disease_label).split(",") if d.strip()]
    in_valid = [norm(d) in valid_nodes_norm for d in diseases]
    return all(in_valid)

df["in_ontology"] = df["disease_label"].apply(check_in_ontology)
df_f = df[df["in_ontology"]].copy()

# 질환별 집계
counts = df_f["disease_label"].value_counts()

print(f"온톨로지 노드 수: {len(valid_nodes)}개")
print(f"온톨로지에 있는 데이터: {len(df_f):,}개 ({len(df_f)/len(df)*100:.1f}%)")
print("="*120)
print(f"{'순위':<5} {'질환명':<60} {'경로(root→node)':<45} {'샘플 수':<10} {'비율'}")
print("="*120)

for i, (disease, cnt) in enumerate(counts.items(), 1):
    pct = cnt / len(df_f) * 100
    # 콤마로 합쳐진 경우 첫 항목을 경로 기준으로 사용
    first = disease.split(",")[0].strip()
    key = norm(first)
    match = sorted(norm_map.get(key, []))
    path = path_to_root(match[0]) if match else []
    print(f"{i:<5} {disease:<60} {path!s:<45} {cnt:<10} {pct:>6.2f}%")
    if i == 50:
        print("\n... (나머지 생략) ...")
        break

print("="*120)


온톨로지 노드 수: 370개
온톨로지에 있는 데이터: 279,150개 (67.6%)
순위    질환명                                                          경로(root→node)                                 샘플 수       비율
1     no definitive diagnosis                                      []                                            102380      36.68%
2     allergic contact dermatitis                                  ['root', 'inflammatory', 'non-infectious', 'Eczema', 'Allergic contact dermatitis'] 47510       17.02%
3     irritated seborrheic keratosis (from "sk/isk")               ['root', 'proliferations', 'malignant', 'non-melanocytic', 'Seborrheic keratosis', 'irritated seborrheic keratosis (from "sk/isk")'] 15906        5.70%
4     psoriasis                                                    ['root', 'inflammatory', 'non-infectious', 'Psoriasis'] 13794        4.94%
5     eczema                                                       ['root', 'inflammatory', 'non-infectious', 'Eczema'] 6808         2.44%
6     basal cell carcino